In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras
import json

In [2]:
businesses = pd.read_csv("Businesses.csv")
income = pd.read_csv("Income.csv")
polling_places = pd.read_csv("PollingPlaces2019.csv")
population = pd.read_csv("Population.csv")
stops = pd.read_csv("Stops.txt")
catchments_future = gpd.read_file("Catchments/catchments_future.shp")
catchments_primary = gpd.read_file("Catchments/catchments_primary.shp")
catchments_secondary = gpd.read_file("Catchments/catchments_secondary.shp")
sa2 = gpd.read_file("SA2_2021_AUST_GDA2020/SA2_2021_AUST_GDA2020.shp")

In [4]:
polling_places = polling_places.dropna(subset=['latitude'])
stops = stops.dropna(subset=['stop_lat'])

In [5]:
polling_places['geom'] = gpd.points_from_xy(polling_places.longitude, polling_places.latitude)
polling_places = polling_places.drop(columns=['latitude', 'longitude','the_geom'])
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops = stops.drop(columns=['stop_lat', 'stop_lat'])

In [6]:
credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [7]:
db, conn = pgconnect(credentials)

Connected successfully.


In [8]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [9]:
srid = 4326
polling_places['geom'] = polling_places['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [10]:
def create_wkt_element(geom, srid):
    if geom is not None:  # Check if geometry is not None
        if geom.geom_type == 'Polygon':
            geom = MultiPolygon([geom])
        return WKTElement(geom.wkt, srid)
    else:
        return None  # Return None if geometry is None

sa2og = sa2.copy()  # creating a copy of the original for later
sa2['geom'] = sa2['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sa2 = sa2.drop(columns="geometry")  # deleting the old copy

catchments_futureog = catchments_future.copy()  # creating a copy of the original for later
catchments_future['geom'] = catchments_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
catchments_future = catchments_future.drop(columns="geometry")  # deleting the old copy

catchments_primaryog = catchments_primary.copy()  # creating a copy of the original for later
catchments_primary['geom'] = catchments_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
catchments_primary = catchments_primary.drop(columns="geometry")  # deleting the old copy

catchments_secondaryog = catchments_secondary.copy()  # creating a copy of the original for later
catchments_secondary['geom'] = catchments_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
catchments_secondary = catchments_secondary.drop(columns="geometry")  # deleting the old copy

In [11]:
sa2 = sa2.dropna(subset=['geom'])

In [12]:
# Remove rows where GCC_NAME21 is not 'Greater Sydney'
sa2.drop(sa2[sa2['GCC_NAME21'] != 'Greater Sydney'].index, inplace=True)

In [13]:
# Get unique SA2_CODE21 values from sa2 GeoDataFrame
valid_sa2_codes = set(sa2['SA2_CODE21'])


In [14]:
businesses = businesses[businesses['sa2_code'].astype(str).str.zfill(9).isin(valid_sa2_codes)]
income = income[income['sa2_code21'].astype(str).str.zfill(9).isin(valid_sa2_codes)]
population = population[population['sa2_code'].astype(str).str.zfill(9).isin(valid_sa2_codes)]

In [15]:
from sqlalchemy import text
create_schema = """DROP SCHEMA IF EXISTS assignment CASCADE;
                    CREATE SCHEMA assignment;"""
create_sa2_regions_table = """
    DROP TABLE IF EXISTS assignment.sa2_data CASCADE;
    CREATE TABLE IF NOT EXISTS assignment.sa2_data (
        id SERIAL PRIMARY KEY,
        sa2_code VARCHAR,
        sa2_name VARCHAR,
        chg_flag INTEGER,
        chg_lbl VARCHAR,
        sa3_code VARCHAR,
        sa3_name VARCHAR,
        sa4_code VARCHAR,
        sa4_name VARCHAR,
        gcc_code VARCHAR,
        gcc_name VARCHAR,
        ste_code VARCHAR,
        ste_name VARCHAR,
        aus_code VARCHAR,
        aus_name VARCHAR,
        areasqkm DOUBLE PRECISION,
        loci_uri VARCHAR,
        geom GEOMETRY(MULTIPOLYGON, 4326)
    );
    """

create_stops_table = """
    DROP TABLE IF EXISTS assignment.stops CASCADE;
    CREATE TABLE IF NOT EXISTS assignment.stops (
        stop_id VARCHAR PRIMARY KEY,
        stop_code VARCHAR,
        stop_name VARCHAR,
        stop_lon FLOAT,
        location_type INTEGER,
        parent_station VARCHAR,
        wheelchair_boarding INTEGER,
        platform_code VARCHAR,
        geom GEOMETRY(POINT, 4326)
    );
    """ 
create_income_table = """
    DROP TABLE IF EXISTS assignment.income CASCADE;
    CREATE TABLE IF NOT EXISTS assignment.income (
        sa2_code INTEGER PRIMARY KEY,
        sa2_name VARCHAR,
        earners INTEGER,
        median_age INTEGER,
        median_income INTEGER,
        mean_income INTEGER
    );
    """ 
create_population_table = """
    DROP TABLE IF EXISTS assignment.population CASCADE;
    CREATE TABLE IF NOT EXISTS assignment.population (
        sa2_code VARCHAR PRIMARY KEY,
        sa2_name VARCHAR,
        "0-4_people" INTEGER,
        "5-9_people" INTEGER,
        "10-14_people" INTEGER,
        "15-19_people" INTEGER,
        "20-24_people" INTEGER,
        "25-29_people" INTEGER,
        "30-34_people" INTEGER,
        "35-39_people" INTEGER,
        "40-44_people" INTEGER,
        "45-49_people" INTEGER,
        "50-54_people" INTEGER,
        "55-59_people" INTEGER,
        "60-64_people" INTEGER,
        "65-69_people" INTEGER,
        "70-74_people" INTEGER,
        "75-79_people" INTEGER,
        "80-84_people" INTEGER,
        "85_and_over_people" INTEGER,
        total_people INTEGER
    );
    """ 
create_business_table = """
    DROP TABLE IF EXISTS assignment.businesses CASCADE;
    CREATE TABLE IF NOT EXISTS assignment.businesses (
        industry_code VARCHAR PRIMARY KEY,
        industry_name VARCHAR,
        sa2_code VARCHAR,
        sa2_name VARCHAR,
        "0_to_50k_businesses" INTEGER,
        "50k_to_200k_businesses" INTEGER,
        "200k_to_2m_businesses" INTEGER,
        "2m_to_5m_businesses" INTEGER,
        "5m_to_10m_businesses" INTEGER,
        "10m_or_more_businesses" INTEGER,
        total_businesses INTEGER
    );
    """ 
create_polling_places_table = """
    DROP TABLE IF EXISTS assignment.polling_places CASCADE;
    CREATE TABLE IF NOT EXISTS assignment.polling_places (
        fid VARCHAR PRIMARY KEY,
        state VARCHAR,
        division_id INTEGER,
        division_name VARCHAR,
        polling_place_id INTEGER,
        polling_place_type_id INTEGER,
        polling_place_name VARCHAR,
        premises_name VARCHAR,
        premises_address_1 VARCHAR,
        premises_address_2 VARCHAR,
        premises_address_3 VARCHAR,
        premises_suburb VARCHAR,
        premises_state_abbreviation VARCHAR,
        premises_post_code VARCHAR,
        geom GEOMETRY(POINT, 4326)
    );
    """ 
create_schema = text(create_schema)
create_sa2_regions_table = text(create_sa2_regions_table)
create_stops_table = text(create_stops_table)
create_income_table = text(create_income_table)
create_population_table = text(create_population_table)
create_business_table = text(create_business_table)
create_polling_places_table = text(create_polling_places_table)

In [16]:
conn.execute(create_sa2_regions_table)
conn.execute(create_stops_table)
conn.execute(create_income_table)
conn.execute(create_population_table)
conn.execute(create_business_table)
conn.execute(create_polling_places_table)

In [17]:
polling_places.to_sql('polling_places', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
sa2.to_sql("sa2", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
population.to_sql('population', conn, if_exists='append', index=False)
income.to_sql('income', conn, if_exists='append', index=False)
businesses.to_sql('businesses', conn, if_exists='append', index=False)

87

In [18]:
conn.commit()
conn.close()

In [19]:
businesses

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
28,A,"Agriculture, Forestry and Fishing",102011028,Avoca Beach - Copacabana,3,0,3,0,0,0,6
29,A,"Agriculture, Forestry and Fishing",102011029,Box Head - MacMasters Beach,8,6,3,0,0,0,17
30,A,"Agriculture, Forestry and Fishing",102011030,Calga - Kulnura,69,43,94,5,3,3,215
31,A,"Agriculture, Forestry and Fishing",102011031,Erina - Green Point,12,8,4,3,0,0,27
32,A,"Agriculture, Forestry and Fishing",102011032,Gosford - Springfield,8,5,6,0,0,0,19
...,...,...,...,...,...,...,...,...,...,...,...
12211,S,Other Services,128021537,Royal National Park,0,0,0,0,0,0,0
12212,S,Other Services,128021538,Sutherland - Kirrawee,21,66,58,3,3,0,152
12213,S,Other Services,128021607,Engadine,13,41,31,3,0,0,87
12214,S,Other Services,128021608,Loftus - Yarrawarrah,0,10,10,0,0,0,22
